In [1]:
import numpy as np
import sensors
import ecm_determ

In [2]:
DIST_RATIO = 0.5

In [3]:
Num_sensors1 = 5
Num_emitters1 = 1
sample_size1 = 100
theta1_rad = [0.5] # Направление прибытия (DOA) в радианах
theta1_deg = np.rad2deg(theta1_rad[0]) # Направление прибытия (DOA) в градусах
#Q1 = 1.1 * np.eye(Num_sensors1, dtype=np.float64) # Ковариация шума
Q1 = sensors.random_complex_cov(n=Num_sensors1, max_real=1.2, seed=50)
A1 = np.exp(-2j * np.pi * DIST_RATIO * np.arange(Num_sensors1).reshape(-1,1) * np.sin(theta1_rad)) # Матрица управляющих векторов
# Генерация векторов сигнала, шума и принятого сигнала
S1 = sensors.gds(Num_emitters1, sample_size1)
n1 = sensors.gss(Num_sensors1, sample_size1, Q1)
X1 = (A1 @ S1.T + n1.T).T

In [4]:
X1_with_mv = sensors.MCAR(X1, [2,4], [40, 40])

In [5]:
EM_theta1_rad, EM_S, EM_Q, lhd_1 = ecm_determ.multi_start_ECM_un(X1_with_mv, 1, num_of_starts=10, max_iter=20, rtol=1e-6)

0-th start
Initial theta = [-2.83488836]
incomplete likelihood is -206.34194327959588 on iteration 0
incomplete likelihood is -112.81415749566568 on iteration 1
incomplete likelihood is -133.4622393836645 on iteration 2
incomplete likelihood is -143.379825485678 on iteration 3
incomplete likelihood is -147.88676989804728 on iteration 4
incomplete likelihood is -145.9511851524127 on iteration 5
incomplete likelihood is -135.19623893119496 on iteration 6
incomplete likelihood is -111.62420743466514 on iteration 7
incomplete likelihood is -66.60083499738974 on iteration 8
incomplete likelihood is 18.450230246771728 on iteration 9
incomplete likelihood is 168.1868413651722 on iteration 10
incomplete likelihood is 324.79783434257115 on iteration 11
incomplete likelihood is 335.0182927976283 on iteration 12
incomplete likelihood is 290.5748500760834 on iteration 13
incomplete likelihood is 250.5945856578108 on iteration 14
incomplete likelihood is 212.14218920711522 on iteration 15
incomplet

In [6]:
EM_theta1_deg = np.rad2deg(EM_theta1_rad)
print(f"Погрешность в смысле разности углов в градусах: {np.abs(EM_theta1_deg-theta1_deg)},")
print(f"Погрешность в смысле разности синусов углов: {np.abs(np.sin(EM_theta1_rad)-np.sin(theta1_rad))}.")

Погрешность в смысле разности углов в градусах: [0.26133515],
Погрешность в смысле разности синусов углов: [0.00400777].
